# 极客时间 - 程序员基础数学课 - [第32课 - 文本聚类](https://time.geekbang.org/column/article/84949)

使用欧氏距离的K-Means聚类算法

首先，我使用 sklearn 库中的 CountVectorizer，对一个测试的文档集合构建特征，也就是词典。这个测试集合有 7 句话，2 句关于篮球，2 句关于电影，还有 3 句关于游戏。具体代码如下：

In [2]:
from sklearn.feature_extraction.text import CountVectorizer

#模拟文档集合
corpus = ['I like great basketball game',
          'This video game is the best action game I have ever played',
          'I really really like basketball',
          'How about this movie? Is the plot great?',
          'Do you like RPG game?',
          'You can try this FPS game',
          'The movie is really great, so great! I enjoy the plot']

#把文本中的词语转换为词典和相应的向量
vectorizer = CountVectorizer()
vectors = vectorizer.fit_transform(corpus)

#输出所有的词条（所有维度的特征）
print('所有的词条（所有维度的特征）')
print(vectorizer.get_feature_names())
print('\n')

#输出(文章ID, 词条ID) 词频
print('(文章ID, 词条ID) 词频')
print(vectors)
print('\n')

所有的词条（所有维度的特征）
['about', 'action', 'basketball', 'best', 'can', 'do', 'enjoy', 'ever', 'fps', 'game', 'great', 'have', 'how', 'is', 'like', 'movie', 'played', 'plot', 'really', 'rpg', 'so', 'the', 'this', 'try', 'video', 'you']


(文章ID, 词条ID) 词频
  (0, 14)	1
  (0, 10)	1
  (0, 2)	1
  (0, 9)	1
  (1, 9)	2
  (1, 22)	1
  (1, 24)	1
  (1, 13)	1
  (1, 21)	1
  (1, 3)	1
  (1, 1)	1
  (1, 11)	1
  (1, 7)	1
  (1, 16)	1
  (2, 14)	1
  (2, 2)	1
  (2, 18)	2
  (3, 10)	1
  (3, 22)	1
  (3, 13)	1
  (3, 21)	1
  (3, 12)	1
  (3, 0)	1
  (3, 15)	1
  (3, 17)	1
  (4, 14)	1
  (4, 9)	1
  (4, 5)	1
  (4, 25)	1
  (4, 19)	1
  (5, 9)	1
  (5, 22)	1
  (5, 25)	1
  (5, 4)	1
  (5, 23)	1
  (5, 8)	1
  (6, 10)	2
  (6, 13)	1
  (6, 21)	2
  (6, 18)	1
  (6, 15)	1
  (6, 17)	1
  (6, 20)	1
  (6, 6)	1




这里，我们希望使用比词频 tf 更好的 tf-idf 机制，TfidfTransformer 可以帮助我们做到这点，代码和注释如下：

In [3]:
from sklearn.feature_extraction.text import TfidfTransformer

#构建tfidf的值
transformer = TfidfTransformer()
tfidf = transformer.fit_transform(vectorizer.fit_transform(corpus))

# 输出每个文档的向量
tfidf_array = tfidf.toarray()
words = vectorizer.get_feature_names()

for i in range(len(tfidf_array)):
    print ("*********第", i + 1, "个文档中，所有词语的tf-idf*********")
    # 输出向量中每个维度的取值
    for j in range(len(words)):
        print(words[j], ' ', tfidf_array[i][j])
    print('\n')

*********第 1 个文档中，所有词语的tf-idf*********
about   0.0
action   0.0
basketball   0.5762001878088874
best   0.0
can   0.0
do   0.0
enjoy   0.0
ever   0.0
fps   0.0
game   0.42760695600759324
great   0.49251681937925895
have   0.0
how   0.0
is   0.0
like   0.49251681937925895
movie   0.0
played   0.0
plot   0.0
really   0.0
rpg   0.0
so   0.0
the   0.0
this   0.0
try   0.0
video   0.0
you   0.0


*********第 2 个文档中，所有词语的tf-idf*********
about   0.0
action   0.33281201025915585
basketball   0.0
best   0.33281201025915585
can   0.0
do   0.0
enjoy   0.0
ever   0.33281201025915585
fps   0.0
game   0.41003731216791683
great   0.0
have   0.33281201025915585
how   0.0
is   0.23614006972816476
like   0.0
movie   0.0
played   0.33281201025915585
plot   0.0
really   0.0
rpg   0.0
so   0.0
the   0.23614006972816476
this   0.23614006972816476
try   0.0
video   0.33281201025915585
you   0.0


*********第 3 个文档中，所有词语的tf-idf*********
about   0.0
action   0.0
basketball   0.4177331617898359
best   0.0
can   0.

最后，我们就可以进行 K 均值聚类了。由于有篮球、电影和游戏 3 个类别，我选择的 K 是 3，并在 KMeans 的构造函数中设置 n_clusters 为 3。

In [4]:
# 由于有篮球、电影和游戏 3 个类别，我选择的 K 是 3，并在 KMeans 的构造函数中设置 n_clusters 为 3
from sklearn.cluster import KMeans

#进行聚类，在我这个版本里默认使用的是欧氏距离
clusters = KMeans(n_clusters=3)
s = clusters.fit(tfidf_array)

#输出所有质心点，可以看到质心点的向量是组内成员向量的平均值
print('所有质心点的向量')
print(clusters.cluster_centers_)
print('\n')

#输出每个文档所属的分组
print('每个文档所属的分组')
print(clusters.labels_)

#输出每个分组内的文档
dict = {}
for i in range(len(clusters.labels_)):
    label = clusters.labels_[i]
    if label not in dict.keys():
        dict[label] = []
        dict[label].append(corpus[i])
    else:
        dict[label].append(corpus[i])
print(dict)

所有质心点的向量
[[6.93889390e-18 1.10937337e-01 0.00000000e+00 1.10937337e-01
  1.55893413e-01 1.76370419e-01 6.93889390e-18 1.10937337e-01
  1.55893413e-01 3.41360072e-01 2.77555756e-17 1.10937337e-01
  6.93889390e-18 7.87133566e-02 1.25140085e-01 1.38777878e-17
  1.10937337e-01 1.38777878e-17 2.77555756e-17 1.76370419e-01
  6.93889390e-18 7.87133566e-02 1.89324393e-01 1.55893413e-01
  1.10937337e-01 2.75807515e-01]
 [2.15328979e-01 0.00000000e+00 0.00000000e+00 0.00000000e+00
  0.00000000e+00 0.00000000e+00 1.70518198e-01 0.00000000e+00
  0.00000000e+00 0.00000000e+00 3.94757865e-01 0.00000000e+00
  2.15328979e-01 2.73770106e-01 0.00000000e+00 3.20286293e-01
  0.00000000e+00 3.20286293e-01 1.41544749e-01 0.00000000e+00
  1.70518198e-01 3.94757865e-01 1.52782347e-01 0.00000000e+00
  0.00000000e+00 0.00000000e+00]
 [0.00000000e+00 0.00000000e+00 4.96966675e-01 0.00000000e+00
  0.00000000e+00 0.00000000e+00 0.00000000e+00 0.00000000e+00
  0.00000000e+00 2.13803478e-01 2.46258410e-01 0.00000000

不过，由于 KMeans 具体的实现可能不一样，而且初始质心的选择也有一定随机性，所以你看到的结果可能稍有不同。